In [7]:
import os, math, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

DATA_DIR = "../data/train"   # adjust ONLY if your images live elsewhere
BATCH    = 32
IMG_SZ   = 224
SEED     = 123
VAL_SPLIT = 0.2
AUTOTUNE = tf.data.AUTOTUNE

print(tf.__version__, keras.__version__)


2.19.1 3.11.3


In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_SZ, IMG_SZ),
    batch_size=BATCH,
    label_mode="binary",
    color_mode="rgb"      
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SZ, IMG_SZ),
    batch_size=BATCH,
    label_mode="binary",
    color_mode="rgb"       
)

# Normalize to [0,1]
norm = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x,y: (norm(x), y), num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.map(lambda x,y: (norm(x), y),   num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)


Found 8005 files belonging to 2 classes.
Using 6404 files for training.
Found 8005 files belonging to 2 classes.
Using 1601 files for validation.


In [12]:
data_augment = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augment")

norm = layers.Rescaling(1./255)

base = tf.keras.applications.MobileNetV2(
    include_top=False, weights="imagenet", input_shape=(IMG_SZ, IMG_SZ, 3)
)
base.trainable = False  # warm-up phase

inputs = keras.Input(shape=(IMG_SZ, IMG_SZ, 3), name="image")
x = norm(inputs)
x = data_augment(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(1e-5))(x)
outputs = layers.Dense(1, activation="sigmoid", name="prob")(x)
model = keras.Model(inputs, outputs, name="catsdogs_efficientnetb0")

loss = keras.losses.BinaryCrossentropy(label_smoothing=0.05)
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=loss,
    metrics=[keras.metrics.AUC(name="auc"), "accuracy",
             keras.metrics.Precision(name="precision"),
             keras.metrics.Recall(name="recall")]
)

ckpt = keras.callbacks.ModelCheckpoint(
    "../models/best_warmup.keras", monitor="val_auc", mode="max",
    save_best_only=True, verbose=1
)
es = keras.callbacks.EarlyStopping(
    monitor="val_auc", mode="max", patience=3, restore_best_weights=True, verbose=1
)

hist_warmup = model.fit(
    train_ds, validation_data=val_ds, epochs=10, callbacks=[ckpt, es]
)


Epoch 1/10
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.5011 - auc: 0.5064 - loss: 0.7549 - precision: 0.5050 - recall: 0.5067
Epoch 1: val_auc improved from None to 0.56402, saving model to ../models/best_warmup.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 40s 186ms/step - accuracy: 0.5064 - auc: 0.5105 - loss: 0.7239 - precision: 0.5098 - recall: 0.5074 - val_accuracy: 0.4878 - val_auc: 0.5640 - val_loss: 0.7536 - val_precision: 0.4878 - val_recall: 1.0000
Epoch 2/10
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5069 - auc: 0.5055 - loss: 0.7104 - precision: 0.5070 - recall: 0.5598
Epoch 2: val_auc did not improve from 0.56402
201/201 ━━━━━━━━━━━━━━━━━━━━ 37s 183ms/step - accuracy: 0.5048 - auc: 0.4999 - loss: 0.7020 - precision: 0.5071 - recall: 0.5834 - val_accuracy: 0.4878 - val_auc: 0.5000 - val_loss: 0.6945 - val_precision: 0.4878 - val_recall: 1.0000
Epoch 3/10
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.5079 - auc: 0.5065 - loss: 0.6953 - precision: 0

In [13]:
base.trainable = True
# unfreeze only the top ~40 layers (tweak 30–80 if you like)
for layer in base.layers[:-40]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(5e-5),  # tiny LR for fine-tune
    loss=loss,
    metrics=[keras.metrics.AUC(name="auc"), "accuracy",
             keras.metrics.Precision(name="precision"),
             keras.metrics.Recall(name="recall")]
)

ckpt_ft = keras.callbacks.ModelCheckpoint(
    "../models/best_finetune.keras", monitor="val_auc", mode="max",
    save_best_only=True, verbose=1
)
es_ft = keras.callbacks.EarlyStopping(
    monitor="val_auc", mode="max", patience=4, restore_best_weights=True, verbose=1
)
rlrop = keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc", mode="max", factor=0.5, patience=2, min_lr=1e-6, verbose=1
)

hist_ft = model.fit(
    train_ds, validation_data=val_ds, epochs=20, callbacks=[ckpt_ft, es_ft, rlrop]
)


Epoch 1/20
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.5868 - auc: 0.6151 - loss: 0.6745 - precision: 0.5846 - recall: 0.6344
Epoch 1: val_auc improved from None to 0.54619, saving model to ../models/best_finetune.keras
201/201 ━━━━━━━━━━━━━━━━━━━━ 52s 233ms/step - accuracy: 0.6146 - auc: 0.6584 - loss: 0.6581 - precision: 0.6161 - recall: 0.6222 - val_accuracy: 0.4878 - val_auc: 0.5462 - val_loss: 0.9614 - val_precision: 0.4878 - val_recall: 1.0000 - learning_rate: 5.0000e-05
Epoch 2/20
200/201 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.6522 - auc: 0.7079 - loss: 0.6328 - precision: 0.6564 - recall: 0.6337
Epoch 2: val_auc did not improve from 0.54619
201/201 ━━━━━━━━━━━━━━━━━━━━ 47s 233ms/step - accuracy: 0.6568 - auc: 0.7129 - loss: 0.6296 - precision: 0.6644 - recall: 0.6430 - val_accuracy: 0.4878 - val_auc: 0.5312 - val_loss: 1.1113 - val_precision: 0.4878 - val_recall: 1.0000 - learning_rate: 5.0000e-05
Epoch 3/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - ac

In [16]:
from sklearn.metrics import f1_score

y_true, y_pred = [], []
for x, y in val_ds:
    p = model.predict(x, verbose=0).ravel()
    y_pred.append(p)
    y_true.append(y.numpy().ravel())
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

ths = np.linspace(0.2, 0.8, 61)
f1s = [f1_score(y_true, (y_pred >= t).astype(int)) for t in ths]
best_t = float(ths[int(np.argmax(f1s))])
print("Best threshold:", best_t)

with open("../models/threshold.txt","w") as f:
    f.write(str(best_t))


Best threshold: 0.25


In [17]:
# Keras 3 export -> SavedModel
model.export("../models/savedmodel_k3")
print("Saved SavedModel")

# (You run the converter from terminal next)


INFO:tensorflow:Assets written to: ../models/savedmodel_k3\assets


INFO:tensorflow:Assets written to: ../models/savedmodel_k3\assets


Saved artifact at '../models/savedmodel_k3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='image')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1740199581424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740199586176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740199588640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740199589344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740199580192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740274930384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740274926512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740274931792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740274929152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740171306864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1740275821456: TensorSpec(shape=()